In [1]:
import numpy as np

def calculate_iqs_unphased(true_genotypes, imputed_dosages):
    # Convert imputed dosages to discrete values
    imputed_discrete = np.round(imputed_dosages).astype(int)

    # Clip the imputed discrete values to be within the range of 0 to 2
    imputed_discrete = np.clip(imputed_discrete, 0, 2)

    # Create a contingency table
    contingency_table = np.zeros((3, 3), dtype=int)

    # Fill the contingency table
    for true_geno, imputed_geno in zip(true_genotypes, imputed_discrete):
        for true_allele, imputed_allele in zip(true_geno, imputed_geno):
            contingency_table[int(true_allele), int(imputed_allele)] += 1

    # Calculate the total number of genotypes
    total_genotypes = np.sum(contingency_table)

    # Calculate observed proportion of agreement (Po)
    observed_agreement = np.trace(contingency_table) / total_genotypes

    # Calculate marginal sums
    row_marginals = np.sum(contingency_table, axis=1)
    col_marginals = np.sum(contingency_table, axis=0)

    # Calculate chance agreement (Pc)
    chance_agreement = np.sum((row_marginals * col_marginals) / (total_genotypes ** 2))

    # Calculate IQS
    if chance_agreement == 1:  # To prevent division by zero in case of perfect chance agreement
        iqs_score = 0
    else:
        iqs_score = (observed_agreement - chance_agreement) / (1 - chance_agreement)

    return iqs_score

# Example usage:
true_genotypes = np.array([[0, 1, 2], [1, 2, 0], [2, 0, 1]])
imputed_dosages = np.array([[0.1, 1.2, 1.9], [1.0, 1.8, 0.3], [2.0, 0.5, 1.4]])

iqs_score = calculate_iqs_unphased(true_genotypes, imputed_dosages)
print(f"IQS Score: {iqs_score}")


IQS Score: 1.0


In [12]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score as sklearn_r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler
import optuna
from matplotlib import pyplot as plt
import os
import csv

# Load and preprocess the data
data_directory = '../../../Data/Filtered_unphased_training_data_union_final/'
start = 1

# Initialize lists to store the performance metrics for each chromosome
r2_scores = []
iqs_scores = []
accuracy_scores = []

# Create folders for saving files
output_folder = "../../../Data/model_results_unphased_all_PRS/neural_network/"
model_folder = output_folder + "models_unphased/"
csv_folder = output_folder + "csv_files/"

os.makedirs(model_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)

for chromosome_number in range(start, 23):
    # Create subfolders for the current chromosome
    chr_model_folder = model_folder + f"chr{chromosome_number}/"
    chr_csv_folder = csv_folder + f"chr{chromosome_number}/"

    os.makedirs(chr_model_folder, exist_ok=True)
    os.makedirs(chr_csv_folder, exist_ok=True)

    file_name = data_directory + f"23AndMe_PRS313_merged_chr{chromosome_number}_matching_combined.parquet"
    data = pd.read_parquet(file_name)

    # Split the data into features and target
    X = torch.tensor(data.filter(regex='^(?!.*PRS313_)').values, dtype=torch.float32)
    y = torch.tensor(data.filter(regex='PRS313_').values, dtype=torch.long)

    print("Total SNPs: ", data.shape[1])
    print("PRS313 SNPs: ", y.shape[1])
    print("Total SNPs used for Training: ", X.shape[1])

    # Normalize the features
    scaler = StandardScaler()
    X = torch.tensor(scaler.fit_transform(X), dtype=torch.float32)
    import joblib

    # Save the scaler to disk
    scaler_filename = f'{chr_model_folder}scaler_chr{chromosome_number}.pkl'
    joblib.dump(scaler, scaler_filename)
    print(f"Scaler saved at: {scaler_filename}")
    # Split the data into train-validation and test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the neural network model
    class NeuralNetwork(nn.Module):
        def __init__(self, input_dim, output_dim, hidden_dims, dropout_rate):
            super(NeuralNetwork, self).__init__()
            layers = []
            prev_dim = input_dim
            for hidden_dim in hidden_dims:
                layers.append(nn.Linear(prev_dim, hidden_dim))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(dropout_rate))
                prev_dim = hidden_dim
            layers.append(nn.Linear(prev_dim, output_dim * 3))
            self.net = nn.Sequential(*layers)

        def forward(self, x):
            out = self.net(x)
            out = out.view(out.size(0), -1, 3)
            return out

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Set the hyperparameters for tuning
    input_dim = X_train_val.shape[1]
    output_dim = y_train_val.shape[1]

    # Define the objective function for Optuna with cross-validation and early stopping
    def objective(trial):
        hidden_layers = trial.suggest_int('hidden_layers', 1, 5)
        hidden_dims = [trial.suggest_int(f'n_units_l{i}', 256, 1028, log=True) for i in range(hidden_layers)]
        dropout_rate = trial.suggest_float('dropout_rate', 1e-11, 0.25)
        learning_rate = trial.suggest_float('learning_rate', 1e-11, 1e-5, log=True)
        patience = 40
        weight_decay = trial.suggest_float('weight_decay', 1e-11, 1e-5, log=True)
        batch_size = trial.suggest_categorical('batch_size', [ 32, 64, 128, 256])
        lr_factor = 0.1
        num_epochs = 300

        model = NeuralNetwork(input_dim, output_dim, hidden_dims, dropout_rate).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        criterion = nn.CrossEntropyLoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=lr_factor, patience=patience, verbose=False)

        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        fold_losses = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_val)):
            X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]
            y_train, y_val = y_train_val[train_idx], y_train_val[val_idx]

            train_dataset = TensorDataset(X_train, y_train)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
            best_val_loss = float('inf')
            counter = 0

            for epoch in range(num_epochs):
                train_loss = 0.0
                for batch_X, batch_y in train_loader:
                    batch_X, batch_y = batch_X.to(device), batch_y.to(device)

                    outputs = model(batch_X)
                    loss = criterion(outputs.transpose(1, 2), batch_y)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    train_loss += loss.item()

                train_loss /= len(train_loader)

                val_dataset = TensorDataset(X_val, y_val)
                val_loader = DataLoader(val_dataset, batch_size=batch_size)

                with torch.no_grad():
                    val_loss = 0.0
                    for batch_X, batch_y in val_loader:
                        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                        outputs = model(batch_X)
                        loss = criterion(outputs.transpose(1, 2), batch_y)
                        val_loss += loss.item()

                    val_loss /= len(val_loader)
                    scheduler.step(val_loss)

                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        counter = 0
                    else:
                        counter += 1

                    if counter >= patience:
                        break

            fold_losses.append(best_val_loss)

        return np.mean(fold_losses)
    
    # Create the "optuna_studies" folder if it doesn't exist
    os.makedirs("optuna_studies_nn", exist_ok=True)

    # Create an Optuna study and optimize the hyperparameters
    study_name = f"unphased_full_23andMe_chr{chromosome_number}_study_nn"
    storage_name = f"sqlite:///optuna_studies_nn/{study_name}.db"

    # Check if the study exists
    current_dir = os.getcwd()
    study_exists = os.path.exists(current_dir + f"/optuna_studies_nn/{study_name}.db")

    if study_exists:
        # Load the existing study
        study = optuna.load_study(study_name=study_name, storage=storage_name)
    else:
        # Create a new study
        study = optuna.create_study(direction='minimize', study_name=study_name, storage=storage_name, load_if_exists=True)

    study.optimize(objective, n_trials=10, n_jobs=-1)

    # Print the best hyperparameters and best value
    print(f"Chr {chromosome_number} - Best hyperparameters: {study.best_params}")
    print(f"Chr {chromosome_number} - Best value: {study.best_value:.4f}")

    # Train the final model with the best hyperparameters and early stopping
    best_hidden_layers = study.best_params['hidden_layers']
    best_hidden_dims = [study.best_params[f'n_units_l{i}'] for i in range(best_hidden_layers)]
    best_dropout_rate = study.best_params['dropout_rate']
    best_learning_rate = study.best_params['learning_rate']
    best_patience = 20
    best_weight_decay = study.best_params['weight_decay']
    best_batch_size = study.best_params['batch_size']
    best_lr_factor = 0.25
    best_num_epochs = 300

    model = NeuralNetwork(input_dim, output_dim, best_hidden_dims, best_dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_learning_rate, weight_decay=best_weight_decay)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=best_lr_factor, patience=best_patience, verbose=False)

    train_dataset = TensorDataset(X_train_val, y_train_val)
    train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)

    best_train_loss = float('inf')
    counter = 0

    for epoch in range(best_num_epochs):
        train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            outputs = model(batch_X)
            loss = criterion(outputs.transpose(1, 2), batch_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        if train_loss < best_train_loss:
            best_train_loss = train_loss
            counter = 0
        else:
            counter += 1

        if counter >= best_patience:
            break

        scheduler.step(train_loss)

    # Save the final model
    model_save_path = chr_model_folder + f'final_model_chr{chromosome_number}.pth'
    torch.save(model.state_dict(), model_save_path)
    print(f"Final model saved at: {model_save_path}")

    # Evaluate the final model on the test set
    with torch.no_grad():
        test_outputs = model(X_test.to(device)).softmax(dim=2)
        test_outputs = test_outputs.argmax(dim=-1)
        test_r2 = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy())
        test_iqs = calculate_iqs_unphased(y_test.cpu().numpy(), test_outputs.cpu().numpy())

        test_accuracy = ((y_test == test_outputs).sum() / (test_outputs.shape[0] * test_outputs.shape[1])).numpy()

        # Append performance metrics to the lists
        r2_scores.append(test_r2)
        iqs_scores.append(test_iqs)
        accuracy_scores.append(test_accuracy)

        # Calculate individual R^2 scores for each SNP
        individual_r2_scores = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy(), multioutput='raw_values')

        # Calculate individual IQS scores for each SNP
        individual_iqs_scores = np.array([calculate_iqs_unphased(y_test.cpu().numpy()[:, i].reshape(-1, 1), test_outputs.cpu().numpy()[:, i].reshape(-1, 1)) for i in range(y_test.shape[1])])

        # Get the names of the SNPs from the original dataframe
        snp_names = data.filter(regex='PRS').columns

        # Save individual R^2 scores to a CSV file
        csv_file = chr_csv_folder + f'individual_r2_scores_chr{chromosome_number}.csv'

        with open(csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'R2 Score'])
            for snp, r2_score in zip(snp_names, individual_r2_scores):
                writer.writerow([snp, r2_score])

        print(f"Individual R^2 scores saved at: {csv_file}")

        # Save individual IQS scores to a CSV file
        iqs_csv_file = chr_csv_folder + f'individual_iqs_scores_chr{chromosome_number}.csv'

        with open(iqs_csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'IQS Score'])
            for snp, iqs_score in zip(snp_names, individual_iqs_scores):
                writer.writerow([snp, iqs_score])

        print(f"Individual IQS scores saved at: {iqs_csv_file}")

    # Create a DataFrame to store the performance metrics for each chromosome
    performance_df = pd.DataFrame({
        'Chromosome': list(range(start, chromosome_number + 1)),
        'R2 Score': r2_scores,
        'IQS Score': iqs_scores,
        'Accuracy Score': accuracy_scores
    })

    # Save the performance metrics to a CSV file
    performance_csv_file = csv_folder + 'performance_metrics.csv'
    performance_df.to_csv(performance_csv_file, index=False)
    print(f"Performance metrics saved at: {performance_csv_file}")

Total SNPs:  2028
PRS313 SNPs:  30
Total SNPs used for Training:  1998
Scaler saved at: ../../../Data/model_results_unphased_all_PRS/neural_network/models_unphased/chr1/scaler_chr1.pkl


/Users/gea2/Documents/medicaid/DeepImpute/.venv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


KeyboardInterrupt: 

In [8]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as sklearn_r2_score
from torchmetrics.classification import BinaryAUROC
import os
import csv
import joblib
import optuna

# Load and preprocess the data
data_directory = '../../../Data/Filtered_unphased_training_data_union_final/'
start = 1

# Initialize lists to store the performance metrics for each chromosome
r2_scores = []
iqs_scores = []
accuracy_scores = []
auc_scores = []

# Folders for saved models and CSV files
output_folder = "../../../Data/model_results_unphased_all_PRS/neural_network/"
model_folder = output_folder + "models_unphased/"
csv_folder = output_folder + "csv_files/"

def safe_auc_metric(y_pred, y_true):
    if y_true.sum() == 0:
        print("All true labels are negative. Returning NaN.")
        return np.NaN
    return auc_metric(y_pred, y_true)

for chromosome_number in range(start, 23):
    # Paths for the current chromosome
    chr_model_folder = model_folder + f"chr{chromosome_number}/"
    chr_csv_folder = csv_folder + f"chr{chromosome_number}/"

    file_name = data_directory + f"23AndMe_PRS313_merged_chr{chromosome_number}_matching_combined.parquet"
    data = pd.read_parquet(file_name)

    # Split the data into features and target
    X = data.filter(regex='^(?!.*PRS313_)').values
    y = data.filter(regex='PRS313_').values

    print("Total SNPs: ", data.shape[1])
    print("PRS313 SNPs: ", y.shape[1])
    print("Total SNPs used for Training: ", X.shape[1])

    # Load the scaler
    scaler_filename = f'{chr_model_folder}scaler_chr{chromosome_number}.pkl'
    scaler = joblib.load(scaler_filename)
    X = torch.tensor(scaler.transform(X), dtype=torch.float32)

    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)

    # Split the data into train-validation and test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the neural network model
    class NeuralNetwork(nn.Module):
        def __init__(self, input_dim, output_dim, hidden_dims, dropout_rate):
            super(NeuralNetwork, self).__init__()
            layers = []
            prev_dim = input_dim
            for hidden_dim in hidden_dims:
                layers.append(nn.Linear(prev_dim, hidden_dim))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(dropout_rate))
                prev_dim = hidden_dim
            layers.append(nn.Linear(prev_dim, output_dim * 3))
            self.net = nn.Sequential(*layers)

        def forward(self, x):
            out = self.net(x)
            out = out.view(out.size(0), -1, 3)
            return out

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load Optuna study
    study_name = f"unphased_full_23andMe_chr{chromosome_number}_study_nn"
    storage_name = f"sqlite:///optuna_studies_nn/{study_name}.db"

    # Check if the study exists
    current_dir = os.getcwd()
    study_exists = os.path.exists(current_dir + f"/optuna_studies_nn/{study_name}.db")

    if study_exists:
        # Load the existing study
        study = optuna.load_study(study_name=study_name, storage=storage_name)
        best_hidden_layers = study.best_params['hidden_layers']
        best_hidden_dims = [study.best_params[f'n_units_l{i}'] for i in range(best_hidden_layers)]
        best_dropout_rate = study.best_params['dropout_rate']
        best_learning_rate = study.best_params['learning_rate']
        best_patience = 20
        best_weight_decay = study.best_params['weight_decay']
        best_batch_size = study.best_params['batch_size']
        best_lr_factor = 0.25
        best_num_epochs = 300

        input_dim = X_train_val.shape[1]
        output_dim = y_train_val.shape[1]

        model = NeuralNetwork(input_dim, output_dim, best_hidden_dims, best_dropout_rate).to(device)

        model_save_path = chr_model_folder + f'final_model_chr{chromosome_number}.pth'
        state_dict = torch.load(model_save_path, map_location=device)

        model.load_state_dict(state_dict, strict=False)
        model.to(device)
        model.eval()

        # Evaluate the final model on the test set
        with torch.no_grad():
            test_outputs = model(X_test.to(device)).softmax(dim=2)
            test_outputs = test_outputs.argmax(dim=-1)
            test_r2 = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy())
            test_iqs = calculate_iqs_unphased(y_test.cpu().numpy(), test_outputs.cpu().numpy())
            test_accuracy = ((y_test == test_outputs).sum() / (test_outputs.shape[0] * test_outputs.shape[1])).numpy()

            # Calculate the AUC for each SNP individually and then average them
            y_test_np = y_test.cpu().numpy()
            test_outputs_np = test_outputs.cpu().numpy()
            auc_metric = BinaryAUROC(thresholds=None)

            individual_aucs = [
                [
                    safe_auc_metric((test_outputs[:, j].round() == i).float(), (y_test[:, j] == i).float())
                    for i in range(3)
                ]
                for j in range(y_test_np.shape[1])
            ]
            # Flatten the list of individual AUCs
            flattened_aucs = [auc for sublist in individual_aucs for auc in sublist]

            # Calculate the mean while ignoring NaN values
            test_auc = np.nanmean(flattened_aucs)
            auc_scores.append(test_auc)

            # Append performance metrics to the lists
            r2_scores.append(test_r2)
            iqs_scores.append(test_iqs)
            accuracy_scores.append(test_accuracy)

            # Calculate individual R^2 scores for each SNP
            individual_r2_scores = sklearn_r2_score(y_test_np, test_outputs_np, multioutput='raw_values')

            # Calculate individual IQS scores for each SNP
            individual_iqs_scores = np.array([calculate_iqs_unphased(y_test_np[:, i].reshape(-1, 1), test_outputs_np[:, i].reshape(-1, 1)) for i in range(y_test_np.shape[1])])

            # Get the names of the SNPs from the original dataframe
            snp_names = data.filter(regex='PRS').columns

            # Save individual R^2 scores to a CSV file
            csv_file = chr_csv_folder + f'individual_r2_scores_chr{chromosome_number}.csv'

            with open(csv_file, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['SNP', 'R2 Score'])
                for snp, r2_score in zip(snp_names, individual_r2_scores):
                    writer.writerow([snp, r2_score])

            print(f"Individual R^2 scores saved at: {csv_file}")

            # Save individual IQS scores to a CSV file
            iqs_csv_file = chr_csv_folder + f'individual_iqs_scores_chr{chromosome_number}.csv'

            with open(iqs_csv_file, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['SNP', 'IQS Score'])
                for snp, iqs_score in zip(snp_names, individual_iqs_scores):
                    writer.writerow([snp, iqs_score])

            print(f"Individual IQS scores saved at: {iqs_csv_file}")

# Create a DataFrame to store the performance metrics for each chromosome
performance_df = pd.DataFrame({
    'Chromosome': list(range(start, 23)),
    'R2 Score': r2_scores,
    'IQS Score': iqs_scores,
    'Accuracy Score': accuracy_scores,
    'AUC Score': auc_scores
})

# Save the performance metrics to a CSV file
performance_csv_file = csv_folder + 'performance_metrics.csv'
performance_df.to_csv(performance_csv_file, index=False)
print(f"Performance metrics saved at: {performance_csv_file}")


Total SNPs:  2028
PRS313 SNPs:  30
Total SNPs used for Training:  1998


/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_9484/3639561551.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


All true labels are negative. Returning NaN.
Individual R^2 scores saved at: ../../../Data/model_results_unphased_all_PRS/neural_network/csv_files/chr1/individual_r2_scores_chr1.csv
Individual IQS scores saved at: ../../../Data/model_results_unphased_all_PRS/neural_network/csv_files/chr1/individual_iqs_scores_chr1.csv
Total SNPs:  1044
PRS313 SNPs:  21
Total SNPs used for Training:  1023


/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_9484/3639561551.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


All true labels are negative. Returning NaN.
Individual R^2 scores saved at: ../../../Data/model_results_unphased_all_PRS/neural_network/csv_files/chr2/individual_r2_scores_chr2.csv
Individual IQS scores saved at: ../../../Data/model_results_unphased_all_PRS/neural_network/csv_files/chr2/individual_iqs_scores_chr2.csv
Total SNPs:  1255
PRS313 SNPs:  16
Total SNPs used for Training:  1239


...

Individual R^2 scores saved at: ../../../Data/model_results_unphased_all_PRS/neural_network/csv_files/chr18/individual_r2_scores_chr18.csv
Individual IQS scores saved at: ../../../Data/model_results_unphased_all_PRS/neural_network/csv_files/chr18/individual_iqs_scores_chr18.csv
Total SNPs:  348
PRS313 SNPs:  7
Total SNPs used for Training:  341


/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_9484/3639561551.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


All true labels are negative. Returning NaN.
Individual R^2 scores saved at: ../../../Data/model_results_unphased_all_PRS/neural_network/csv_files/chr19/individual_r2_scores_chr19.csv
Individual IQS scores saved at: ../../../Data/model_results_unphased_all_PRS/neural_network/csv_files/chr19/individual_iqs_scores_chr19.csv
Total SNPs:  108
PRS313 SNPs:  4
Total SNPs used for Training:  104


/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_9484/3639561551.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


KeyboardInterrupt: 